# Previous Work

#### T5
https://research.google/blog/exploring-transfer-learning-with-t5-the-text-to-text-transfer-transformer/


#### MLOps MLflow: How to install  a Mlflow Tracking Server in docker containers
- Video English https://youtu.be/TDOhzidiJ-A
- Video Spanish https://youtu.be/xMK1QZmS-_w
- Code https://github.com/olonok69/LLM_Notebooks/tree/main/mlflow/MLOPS_mlflow

#### MLOps MLFlow: Fine tune Flan-T5  for text classification
- Video English https://youtu.be/1fUX7U5WJ_M
- Video Spanish https://youtu.be/po1Ct2-jvco
- Code https://github.com/olonok69/LLM_Notebooks/blob/main/t5/T5ForSequenceClassification_custom_mlflow_fine_tuning.ipynb

#### Custom Classifier
![title](general_arch.png)

#### Architecture
![title](custom_classifier.png)


## Requirements
```
mlflow==2.13.1
pymysql
nltk
pandas
packaging>=23.0
pandas>=1.3.5
pip>=22.0.4
PyYAML>=6.0
requests>=2.25.1
fastapi>=0.68.0
pydantic>=1.8.0
uvicorn>=0.15.0
python-dotenv==1.0.0
detectaicore>=0.0.23
evaluate
bitsandbytes
accelerate
datasets
transformers==4.39.3
python-magic
torchvision
PyMySQL
orjson
```


## DockerFile Custom Classifier
```
# Use an official Python runtime as a parent image
FROM python:3.11-slim

RUN apt-get update && \
    apt-get install -y python3-pip  ffmpeg libsm6 libxext6 git net-tools  python3-magic nano iputils-ping procps && \
    rm -rf /var/lib/apt/lists/* /var/cache/apt/archives/*
# Set the working directory to /app
WORKDIR /custom_classifier

# Copy the requirements file into the image
COPY ./requirements.txt /custom_classifier/requirements.txt
# Install any needed packages specified in requirements.txt
RUN pip install -r requirements.txt

# Copy every content from the local folder to the image
COPY . .

EXPOSE 5020
# Run server
CMD ["uvicorn", "endpoint_custom_classifier:endpoint", "--host=0.0.0.0", "--port=5020"]
```

## Docker Compose MLFLOW
```
FROM python:3.11-slim

ARG mlflow_user

# Use the build argument to set an environment variable
ENV USER_MLFLOW=${mlflow_user}

# You can now use the USER_MLFLOW environment variable in your Dockerfile
RUN echo "Mlflow user is: ${USER_MLFLOW}"

RUN apt-get update && apt-get -y upgrade \
    && apt-get install -y libsm6 libxext6 git net-tools  python3-magic nano iputils-ping procps \
    && pip install --upgrade pip \
    && pip --version

RUN apt-get update && apt-get install -y procps \
    && rm -rf /var/lib/apt/lists/*

RUN groupadd ${USER_MLFLOW} && useradd --create-home -g ${USER_MLFLOW} ${USER_MLFLOW}
ENV PATH /home/${USER_MLFLOW}/.local/bin:${PATH}

WORKDIR /home/${USER_MLFLOW}/mlflow

COPY ./mlflow/requirements.txt .

RUN pip install --no-cache-dir -r requirements.txt \
    && rm requirements.txt
RUN mkdir /home/${USER_MLFLOW}/mlflow/mlruns

RUN chown -R ${USER_MLFLOW}:${USER_MLFLOW} /home/${USER_MLFLOW}/mlflow 
RUN chown -R ${USER_MLFLOW}:${USER_MLFLOW} /home/${USER_MLFLOW}/mlflow/mlruns 
RUN chmod -R 777 /home/${USER_MLFLOW}

USER ${USER_MLFLOW}

EXPOSE 5000

CMD mlflow server --backend-store-uri ${BACKEND_STORE_URI} --default-artifact-root ${DEFAULT_ARTIFACT_ROOT} --artifacts-destination ${DEFAULT_ARTIFACTS_DESTINATION} --no-serve-artifacts --host 0.0.0.0 --port 5000
```

### Docker Compose File
```
services:
  mysql:
    image: mariadb:10.3
    container_name: mariadb
    hostname: mariadb
    env_file:
      - .env
    ports:
      - ${MYSQL_PORT1}:3306
      - ${MYSQL_PORT2}:33060
    environment:
      - MYSQL_ROOT_PASSWORD_FILE=/run/secrets/password1
      - MYSQL_PASSWORD=${MYSQL_PASSWORD}
      - MYSQL_DATABASE=${MYSQL_DATABASE}
      - MYSQL_USER=${MYSQL_USER}
    secrets:
      - password1
      - password2
    volumes:
      - database_volume:/var/lib/mysql
    networks:
      - network

  phpmyadmin:
    image: phpmyadmin:latest
    container_name: phpmyadmin
    hostname: phpmyadmin
    depends_on:
      - mysql
    env_file:
      - .env
    environment:
      PMA_HOST: mysql
      MYSQL_ROOT_PASSWORD_FILE: /run/secrets/password1
    secrets:
      - password1
    ports:
      - ${PHPMYADMIN_PORT}:80
    networks:
      - network

  mlflow:
    image: acrdetectaideveastus.azurecr.io/detectai-mlflow:latest
    container_name: mlflow_tracker
    hostname: mlflow_tracker
    depends_on:
      - mysql
    env_file:                                
      - .env
      - .secrets/env-secrets
    ports:
      - ${MLFLOW_PORT}:5000
    volumes:
      - /home/${USER_MLFLOW}/mlflow/mlruns:/home/${USER_MLFLOW}/mlflow/mlruns
    networks:
      - network

  custom_classifier:
    container_name: custom_classifier
    image: acrdetectaideveastus.azurecr.io/detectai-custom_classifier:latest
    hostname: custom_classifier        
    ports:
      - 5020:5020
    environment:
      - MYSQL_PASSWORD=${MYSQL_PASSWORD}
    depends_on:
      - mysql
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: 1
              capabilities: [ gpu ]      
    volumes:
      - /home/${USER_MLFLOW}/data/dataset:/custom_classifier/dataset
      - ../mlflow/mlruns:/home/${USER_MLFLOW}/mlflow/mlruns
    restart: always
    networks:
      - network

  nginx:
    container_name: nginx
    image: nginx:1.25.1
    volumes:
      - ../docker/configs/nginx/nginx.conf:/etc/nginx/nginx.conf
      - ../docker/configs/nginx/certs:/etc/nginx/certs
    ports:
      - "80:80"
      - "443:443"
    depends_on:
      - mysql
      - phpmyadmin
      - mlflow
      - custom_classifier
    restart: always
    networks:
      - network

volumes:
  database_volume:

secrets:
  password1:
    file: ./.secrets/mysql-root-password.txt
  password2:
    file: ./.secrets/mysql-mlflowuser-password.txt

networks:
  network:
```


## Azure-Pipelines.yml
```
# Docker
# Build and push an image to Azure Container Registry
# https://docs.microsoft.com/azure/devops/pipelines/languages/docker

trigger:
  - develop

resources:
  - repo: self

variables:

  # agents
  azure_build_Agent: "Azure Pipelines"
  hosted_build_Agent: "Default"

  # Container registry service connection established during pipeline creation
  dockerRegistryServiceConnection: "Changeme"
  imageRepository: "Changeme"
  containerRegistry: "Changeme.azurecr.io"
  tag: "$(Build.BuildId)"

  # docker file paths
  dockerfilePath_custom_classifier: "$(Build.SourcesDirectory)/custom_classifier/Dockerfile"
  dockerfilePath_mlflow: "$(Build.SourcesDirectory)/mlflow/Dockerfile"

  # image Repositories
  imageRepository_custom_classifier: "$(imageRepository)-custom_classifier"
  imageRepository_mlflow: "$(imageRepository)-mlflow"

  # Build Variables
  disable_tests: "false" # Set to 'true' to skip the unit tests
  use_no_cache: "false" # Set to 'true' to enable --no-cache which will build the docker images without cache

  # Sonar Cloud
  SonarCloud_Organization: "Changeme"
  SonarCloud_ProjectName: "Changeme-AI-Custom-Classifier"
  SonarCloud_ProjectKey: "Changeme-AI-Custom-Classifier"

  mlflow_user: "Changeme"

stages:
  - stage: Publish_Docker_Artifacts
    displayName: Sonar Analyze + Publish Artifacts
    jobs:
      - job: Publish_Docker_Artifacts
        displayName: Publish Docker Artifacts
        pool: $(azure_build_Agent)
        steps:
          - checkout: self
            fetchDepth: 0

          - task: SonarCloudPrepare@1
            continueOnError: false
            inputs:
              SonarCloud: 'SonarCloud'
              organization: '$(SonarCloud_Organization)'
              scannerMode: 'CLI'
              configMode: 'file'
              extraProperties: |
                sonar.projectKey=$(SonarCloud_ProjectKey)
                sonar.organization=$(SonarCloud_Organization)
                
                # This is the name and version displayed in the SonarCloud UI.
                #sonar.projectName=Detect-AI-Custom-Classifier
                #sonar.projectVersion=1.0


                # Path is relative to the sonar-project.properties file. Replace "\" by "/" on Windows.
                #sonar.sources=.

                # Encoding of the source code. Default is default system encoding
                #sonar.sourceEncoding=UTF-8
              
          - task: SonarCloudAnalyze@1
            continueOnError: false

          - task: SonarCloudPublish@1
            continueOnError: false
            inputs:
              pollingTimeoutSec: '300'

          - task: PublishBuildArtifacts@1
            displayName: Publish the Artifact Bundle
            inputs:
              PathtoPublish: "$(System.DefaultWorkingDirectory)/docker"
              ArtifactName: "docker"
              publishLocation: "Container"              

  - stage: Docker_mlflow_tracker
    displayName: Docker mlflow tracker
    jobs:
      - job: Docker_Image_Builds_mlflow_tracker
        steps:
        - script: |
            docker build \
              --file $(dockerfilePath_mlflow) \
              --build-arg mlflow_user=$(mlflow_user) \
              -t acrdetectaideveastus.azurecr.io/$(imageRepository_mlflow):$(tag) \
              -t acrdetectaideveastus.azurecr.io/$(imageRepository_mlflow):latest .
          displayName: 'Build Docker_mlflow_tracker with build args'

        - task: Docker@2
          displayName: Push Docker Mlflow Tracker
          condition: and(succeeded(), ne(variables['Build.Reason'], 'PullRequest'))
          inputs:
            command: 'push'          
            repository: $(imageRepository_mlflow)            
            containerRegistry: $(dockerRegistryServiceConnection)
            tags: |
              $(tag)
              latest    

  - stage: Docker_custom_classifier
    displayName: Docker Custom Classifier
    jobs:
      - job: Docker_Image_Builds_Custom_Classifier
        displayName: Build and Push Docker Custom Classifier
        pool: $(azure_build_Agent)
        steps:
          - task: Docker@2
            displayName: Build and Push
            inputs:
              command: buildAndPush
              repository: $(imageRepository_custom_classifier)
              dockerfile: $(dockerfilePath_custom_classifier)
              buildArguments: $[format('--no-cache={0}', variables['use_no_cache'])]
              containerRegistry: $(dockerRegistryServiceConnection)
              tags: |
                $(tag)
                latest            
                
                
```